## 一个类CSDN的内容社区，应该怎么通过用户行为，刻画消费用户的画像？


## 选题方向：如何提升留存？

### 准备python包

In [1]:
import numpy as np
import pandas as pd
print('加载numpy和pandas')

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print('加载可视化需要的包')

import warnings
warnings.filterwarnings('ignore')

print('准备完成！')

加载numpy和pandas
加载可视化需要的包
准备完成！


### 读取数据

In [2]:
df = pd.read_csv('/home/mw/input/realworlddata8695/实战数据集.csv')
df.bhv_ts = pd.to_datetime(df.bhv_ts)
df.head()

,idx,user_id,bhv_ts,action_name,target_id,target_category
0,1,5A15d8dd71710055601be6d129066A4D,2020-05-19 13:24:17.775,view,27BA769d9a7100f002b295e4d6068D91,5532C32
1,2,B1A4f1df7f6300ce62e9c82d6ff5985D,2020-04-07 19:12:52.584,view,823980d4475100ded474818a160687B2,5364B4F
2,3,AF33d2f861a300bfc85f805cd306215E,2020-04-27 09:22:18.937,view,B1A632f5cb51003c3d36620cc3066318,5043432
3,4,95A7f85ea4710085064a73d55f0691C3,2020-09-15 21:39:13.934,view,91307a082b63001a32828210f0e5633D,5362102
4,5,C2BC493dcfc200936fe38af22bf5C03C,2020-04-29 20:24:26.399,view,A91A3741b2d2008a4d89f81847e55A7E,5000000


### 数据初探

日志数据一共包含6列，分别代表：
* idx：序号
* user_id：用户ID，用户独立标识 
* bhv_ts：行为时间，用户行为发生时间戳，可以看到**数据的年份是2020年**
* action_name：用户行为的名称；view：浏览；reward：打赏；buy：购买（一般是对资源的购买）；share：分享；mark：收藏；vote：点赞；comp：参与活动
* target_id：用户发生行为的对象的ID，如用户阅读文章的文章ID
* target_category：用户发生行为的对象的所属类别，如用户阅读的文章的所属类别。本字段数字不超过7位数，首位数是对象所属的大类；后面6位中，每两位代表一个小类别，最多会有3个小类别，小类别并列是并列关系。注意：若action_name为reward，则本字段的数字代表reward的具体金额


In [3]:
print('日志数据共有：',df.shape[0],'行')

日志数据共有： 1329805 行


In [4]:
print('验证一下，数据是否存在重复记录，重复记录数：')
df.duplicated(subset = ['user_id','bhv_ts',  ]).sum()

验证一下，数据是否存在重复记录，重复记录数：


0

日志数据的各列均没有缺失值，除了bhv_ts已经被转换为时间类型，其他内容列均为object类



In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1329805 entries, 0 to 1329804
Data columns (total 6 columns):
idx                1329805 non-null int64
user_id            1329805 non-null object
bhv_ts             1329805 non-null datetime64[ns]
action_name        1329805 non-null object
target_id          1329805 non-null object
target_category    1329805 non-null object
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 60.9+ MB


In [18]:
print('最早的时间：',df.bhv_ts.min())
print('最晚的时间：',df.bhv_ts.max())
print('区间包含天数：',df.bhv_ts.max()-df.bhv_ts.min())

最早的时间： 2020-03-01 00:00:07.346000
最晚的时间： 2020-09-30 23:57:47.149000
区间包含天数： 213 days 23:57:39.803000


查看不同action对应的target对象是类型特点发现：
* reward（打赏）和comp（参与活动）的target_category代码都只有3位
* 其他action类型对象的类型代码长度均为7位

In [7]:
for act in df.action_name.unique(): 
    target_cat_len = df[df.action_name==act].target_category.str.len().unique()
    print(act,'行为对应的target category代码长度:',target_cat_len)

view 行为对应的target category代码长度: [7]
mark 行为对应的target category代码长度: [7]
share 行为对应的target category代码长度: [7]
vote 行为对应的target category代码长度: [7]
reward 行为对应的target category代码长度: [3]
comp 行为对应的target category代码长度: [3]
buy 行为对应的target category代码长度: [7]


### 数据加工

根据数据各字段的介绍，可以进一步增加字段列，以方便后续的分析
同时生成原始日志数据的聚合统计表，user_id是唯一的

In [ ]:
#code_list = []
df['target_cat_spe'] = None
for i in range(len(df)):
    cat_code1 = df.target_category[i][1:3]
    cat_code2 = df.target_category[i][3:5]
    cat_code3 = df.target_category[i][5:]
    code = [cat_code1,cat_code2,cat_code3]
    code_list += code
#    print(cat_code1,cat_code2,cat_code3)
#    print(code)
#    print(code_list)
    df['target_cat_spe'][i] = code
code_list = pd.Series(code_list).unique()

In [3]:
df['date'] = df.bhv_ts.dt.date
df['time'] = df.bhv_ts.dt.time
df['month'] = df.bhv_ts.dt.month
df['day'] = df.bhv_ts.dt.day
df['dayofperiod'] = df.bhv_ts.dt.dayofyear
df['dayofperiod'] = df['dayofperiod'] - df['dayofperiod'].min()
df['weekday'] = df.bhv_ts.dt.weekday #0-6对应星期一到星期天
df['isweekend'] = df['weekday'].apply(lambda x: 1 if x>=5 else 0)
df['weekofperiod'] = df.bhv_ts.dt.weekofyear
df['weekofperiod'] = df['weekofperiod'] - df['weekofperiod'].min()
df['hour'] = df.bhv_ts.dt.hour
df['target_cat'] = df.target_category.str[0]
df['target_cat_spe'] = df.target_category.str[1:3]
df['act_target'] = df.action_name + '_' + df.target_cat


In [4]:
# 按user聚合统计
aggs = {
    'dayofperiod':['nunique','max','min',np.ptp],
    'action_name':['nunique'],
    'target_cat':['nunique'],
    'act_target':['nunique','count']
}
agg_df = df.groupby('user_id').agg(aggs)
agg_df.columns = ['df'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df = agg_df.reset_index()
agg_df['act_perc'] = agg_df.df_dayofperiod_nunique/(agg_df.df_dayofperiod_ptp+1)

In [5]:
# 在原始数据上增加新列：new/old
# 第一次出现的user_id为new,其他日期再出现则为old
df = df.merge(agg_df[['user_id','df_dayofperiod_min']])
df['active_day'] = df.dayofperiod - df.df_dayofperiod_min
df['isnew'] = df['active_day'].apply(lambda x: 'new' if x==0 else 'old')

df['active1'] = df['active_day'].apply(lambda x: 1 if x==1 else 0)
df['active7'] = df['active_day'].apply(lambda x: 1 if x==7 else 0)
df['active14'] = df['active_day'].apply(lambda x: 1 if x==14 else 0)
df['active21'] = df['active_day'].apply(lambda x: 1 if x==21 else 0)
df['active30'] = df['active_day'].apply(lambda x: 1 if x==30 else 0)
df['active60'] = df['active_day'].apply(lambda x: 1 if x==60 else 0)


In [6]:
# 活跃天数大于1的用户id
suc_id = agg_df[agg_df.df_dayofperiod_nunique > 1].user_id

In [7]:
df_user = df.groupby(['user_id'])['active1', 'active7', 'active14', 'active21', 'active30', 'active60'].nunique()-1
df_user = df_user.reset_index()
df_user = df_user.merge(agg_df[['user_id','df_dayofperiod_min']],on='user_id',how='left')\
.merge(df[['date','dayofperiod']].drop_duplicates(),left_on='df_dayofperiod_min',right_on='dayofperiod')
# df_user

In [8]:
df.columns

Index(['idx', 'user_id', 'bhv_ts', 'action_name', 'target_id',
       'target_category', 'date', 'time', 'month', 'day', 'dayofperiod',
       'weekday', 'isweekend', 'weekofperiod', 'hour', 'target_cat',
       'target_cat_spe', 'act_target', 'df_dayofperiod_min', 'active_day',
       'isnew', 'active1', 'active7', 'active14', 'active21', 'active30',
       'active60'],
      dtype='object')

In [9]:
agg_df.columns

Index(['user_id', 'df_dayofperiod_nunique', 'df_dayofperiod_max',
       'df_dayofperiod_min', 'df_dayofperiod_ptp', 'df_action_name_nunique',
       'df_target_cat_nunique', 'df_act_target_nunique', 'df_act_target_count',
       'act_perc'],
      dtype='object')

## 定义“留存用户”

**活跃**：一段时间内进入过内容社区进行各类操作的用户，均为”活跃用户“。
假设本次分析数据中用户的首条记录时间即是首次使用时间。

In [10]:
# 活跃天数超过1天的留存用户日志数据表
df_suc = df[df.user_id.isin(suc_id)]


In [29]:
a = df.groupby('active_day')['user_id'].nunique()
b = df_suc.groupby('active_day')['user_id'].nunique()
df_ab = pd.concat([a,b],axis=1)
df_ab.columns = ['总体用户','留存用户']
plt.figure(figsize=(15,10))
df_ab.head(15).plot(kind='bar',color=['tab:blue','C3'])
plt.xlabel('首登录后的活跃日期')
plt.title('用户数量')
plt.show()

<Figure size 1080x720 with 0 Axes>

<Figure size 432x288 with 1 Axes>

根据用户首次进入社区后是否仍有后续活跃，可以将用户分成 **一次性用户** 和 **留存用户**。
从上图可以看到，整体用户基盘中，存在大量的一次性用户。

* 一次性用户，只在首次活跃的日期有行为动作，后面再没有使用行为数据。
* 留存用户，在首次使用日期后仍有其他日期有活跃行为。

将行为发生日期作为基础，社区平台的**新增用户**群体是一个非常重要的划分维度。
应用上面的定义方法，新增用户群体在周期性复盘时也可以划分为一次性用户和留存用户探究其内部结构。

In [11]:
# dt：新增用户中一次性用户与留存用户的数量
dt = df[df.isnew=='new'][['date','user_id']].drop_duplicates().merge(agg_df[['user_id','df_dayofperiod_nunique']])
dt['new_type'] = dt.df_dayofperiod_nunique.apply(lambda x: 1 if x > 1 else 0)
dt = dt.groupby(['date','new_type']).user_id.nunique().reset_index().pivot_table(index='date',columns='new_type')
dt.columns=['nsuc','suc']
dt['ttl'] = dt.suc + dt.nsuc
dt['month'] = pd.to_datetime(dt.index.values).month
# dt

In [56]:
plt.figure(figsize=(10,6))
dt.iloc[7:-7].groupby('month').sum().plot(colors=['green','C3','tab:blue'])
plt.legend(['一次性用户','留存用户','新增用户总体'])
plt.title('新增用户每月数量')

Text(0.5, 1.0, '新增用户每月数量')

<Figure size 720x432 with 0 Axes>

<Figure size 432x288 with 1 Axes>

从上图来看，从3月至9月期间，社区的新增用户数据经历了先升高再降低的变化。
* 第二季度各月的新增用户处于良好水平。值得注意的是，5月的最高值主要是由于一次性用户激增造成的，猜测是平台搞了什么活动，可进一步验证是否存在类似的“薅羊毛”的现象。
* 第三季度，新增用户逐月下降，且8月开始一次性用户占比已经超过留存用户，这是一个非常危险的信号，长期持续会造成用户基盘萎缩。



**PV**：某时间段内，各类型行为所有的发生数

**UV**：某时间段内，做了任何行为的活跃用户数


In [57]:
plt.figure(figsize=(8,6))
df[df.isnew=='old'].groupby('date')['user_id'].count().plot()
df[df.isnew=='new'].groupby('date')['user_id'].count().plot()
plt.legend(['老用户','新用户'])
plt.title('PV')
plt.show()

plt.figure(figsize=(8,6))
df[df.isnew=='old'].groupby('date')['user_id'].nunique().plot()
df[df.isnew=='new'].groupby('date')['user_id'].nunique().plot()
plt.legend(['老用户','新用户'])
plt.title('UV')
plt.show()


<Figure size 576x432 with 1 Axes>

<Figure size 576x432 with 1 Axes>

## 用户留存分析

从社区的PV来看，主要的活跃操作还是由留存后的老用户带来的。

下面看看留存用户在后续不同时间周期内的活跃情况：

In [12]:
# active_df：留存用户的活跃情况
active_df = (df_suc.groupby(['df_dayofperiod_min','user_id'])['active1', 'active7',\
 'active14', 'active21','active30', 'active60'].nunique()-1).reset_index()\
.groupby('df_dayofperiod_min')['active1', 'active7', 'active14',  'active21','active30', 'active60'].sum()
active_df['suc'] = dt['suc'].values[:-1]
# active_df

In [84]:
peret = []
for i in [1,7,14,21,30,60]:
    cl_name = 'active'+str(i)
    perc = (active_df.iloc[7:-i][cl_name]/active_df.iloc[7:-i].suc).mean().round(3)
    print(i,'天留存率平均值：',perc)
    peret.append(perc)

1 天留存率平均值： 0.454
7 天留存率平均值： 0.155
14 天留存率平均值： 0.1
21 天留存率平均值： 0.069
30 天留存率平均值： 0.045
60 天留存率平均值： 0.027


In [86]:
print('留存率周降幅：')
for i in range(len(peret)-2):
    print(i+1,'周降幅：',((peret[i]-peret[i+1])/peret[i]).round(3))

留存率周降幅：
1 周降幅： 0.659
2 周降幅： 0.355
3 周降幅： 0.31
4 周降幅： 0.348


从留存用户的不同时间留存率看：
* 近日留存率的水平最高，接近50%，平均值 0.454
* 一周后的7天留存率就已经下降至 15%水平，后期的留存平均每周下降 30%左右

由此看来，抓住用户，促成长期留存的关键就在**第一周**内

In [90]:
cols = active_df.columns.to_list()[:-1]

print('留存用户的留存率趋势')
plt.figure(figsize=(10,8))
for i in range(len(cols)):
    (active_df[cols[i]]/active_df.suc)[7:-7].plot(alpha=0.5)
plt.legend(['1天留存','7天留存','14天留存','21天留存','30天留存','60天留存'])
plt.show()
print('----------------------------------------------------------------------------------')
plt.figure(figsize=(20,10))
# fig,ax = plt.subplots(2,3)
# ax.plot()
for i in range(len(cols)):
    plt.subplot(2, 3, i+1)
    (active_df[cols[i]]/active_df.suc)[7:-7].plot(color='C4',alpha=0.7)
    # ax_[i].plot(active_df[cols[i]]/active_df.suc,color='C4')
    plt.xlabel('新增日期')
    # plt.xticks(pd.date_range(start='2020-03-01',periods=214,freq='D').to_list())
    plt.title('Index: {}'.format(cols[i]) )
plt.show()


留存用户的留存率趋势


<Figure size 720x576 with 1 Axes>

----------------------------------------------------------------------------------


<Figure size 1440x720 with 6 Axes>

从各留存率曲线的趋势来看，近3个月（第三季度7-9月）新增的用户的留存率反而呈现轻微的增长，再结合近三个月新增用户数量的减少，可以认为最近一个季度的新增用户更属于自然增长的用户，这类用户的自驱力更强，可以当作社区活跃的中流砥柱。
对这类用户行为的更深挖掘，可以确立社区的定位亮点。

## 抓取焦点用户

In [13]:
# 按留存user聚合统计
agg_df_suc = agg_df[agg_df.user_id.isin(suc_id)]

通过留存用户的生命周期(df_dayofperiod_ptp)与活跃天数(df_dayofperiod_nunique)的联合分布情况，可以发现：
* 留存用户的生命周期，基本在一个月左右的水平
* 留存用户的活跃天数更为集中，由于生命周期长度的限制，活跃天数在低数量区域内集聚

In [97]:
plt.figure(figsize=(8,8))
sns.jointplot(x=agg_df_suc.df_dayofperiod_ptp+1, y=agg_df_suc.df_dayofperiod_nunique, kind="kde")

<Figure size 576x576 with 0 Axes>

<Figure size 432x432 with 3 Axes>

尝试查看不同生命周期长度用户的**“活跃天数比例”** 这一指标的特征：
* 生命长度在一周内时，活跃天数的比例还是分散的
* 生命周期大于一周，留存用户的活跃情况就开始呈现一种稳定的模式：活跃天数比例的贬值在 0.2 左右，且随着生命周期的延长，峰值水平逐渐降低


In [167]:
# 不同生命周期长度用户的活跃天数比例
plt.figure(figsize=(18,10))
slots = [3,7,14,21,30,60]
for i in range(len(slots)):
    plt.subplot(2,3,i+1)
    if i == 0:
        my_list = agg_df_suc[agg_df_suc.df_dayofperiod_ptp+1<=7]
        # print(my_list.head())
        my_list = my_list.df_dayofperiod_nunique/(my_list.df_dayofperiod_ptp+1)
        sns.distplot(a=my_list[my_list<1], kde=True,color='C3')
        plt.xlabel('活跃天数占比')
        plt.title('生命周期小于一周的留存用户的活跃天数比例密度图')
    else:
        my_list = agg_df_suc[agg_df_suc.df_dayofperiod_ptp+1<=slots[i]][agg_df_suc.df_dayofperiod_ptp+1>slots[i-1]]
        # print(my_list.head())
        my_list = my_list.df_dayofperiod_nunique/(my_list.df_dayofperiod_ptp+1)
        sns.distplot(a=my_list[my_list<=1], kde=True)
        plt.xlabel('活跃天数占比')
        plt.title('{}-{}天生命周期留存用户的活跃天数比例密度图'.format(slots[i-1]+1,slots[i]))
plt.show()

<Figure size 1296x720 with 6 Axes>

### 定位焦点用户类型进行深入分析

- 第一类用户，短生命周期 - 高活跃频率：生命周期大于7天小于60天，活跃天数比例大于0.5
- 第二类用户，长生命周期 - 较高活跃频率：生命周期大于60天，活跃天数比例大于0.2


In [180]:
plt.figure(figsize=(12,8))
sns.scatterplot(x=agg_df_suc.df_dayofperiod_ptp+1,\
y=agg_df_suc.df_dayofperiod_nunique/(agg_df_suc.df_dayofperiod_ptp+1),alpha=0.3)
# hue=agg_df_suc.df_act_target_nunique,
plt.xlim(8,214)
plt.axvline(x=60,ls='--',color='C3')
plt.axhline(y=0.2,ls='--',color='green')
plt.axhline(y=0.5,ls='--',color='green')
plt.xlabel('生命周期长度')
plt.ylabel('活跃天数比例')
plt.title('留存用户的活跃情况')

Text(0.5, 1.0, '留存用户的活跃情况')

<Figure size 864x576 with 1 Axes>

那么，两类焦点用户群体，与留存用户总体相比是否有什么独特的行为模式吗？
接下来，进一步分析两种类型的用户在进入社区的初期（首周及首日）行为特征

## 核心用户分析初期行为分析

In [14]:
# 提取两类目标群体的user_id
sub1_id = agg_df_suc[agg_df_suc.df_dayofperiod_ptp.isin([14,60])][agg_df_suc.act_perc>0.5].user_id
sub2_id = agg_df_suc[agg_df_suc.df_dayofperiod_ptp>=60][agg_df_suc.act_perc>0.2].user_id

In [15]:
df_act7 = df[df.active_day<7]
df_act1 = df[df.active_day<1]

In [32]:
# 定义注册后第一周内的聚合统计表
agg_act7 = (df_act7.pivot_table(index='user_id',columns='act_target',values='target_id',aggfunc=['count'])/7).fillna(0)
agg_act7.columns = pd.Series(df_act7.act_target.unique()).sort_values()
agg_act7 = agg_act7.reset_index()
agg_act7 = agg_act7.merge(agg_df[['user_id','df_dayofperiod_ptp','act_perc']])

aggs = {
    'dayofperiod':['count','nunique'],
    'action_name':['nunique'],
    'target_cat':['nunique'],
    'act_target':['nunique']
}
agg_df_p = df_act7.groupby('user_id').agg(aggs)
agg_df_p.columns = ['_'.join(col).strip() for col in agg_df_p.columns.values]
agg_df_p = agg_df_p.reset_index()
agg_df_p['w_act_perc'] = agg_df_p.dayofperiod_nunique/7
agg_df_p['dayofperiod_count'] = agg_df_p.dayofperiod_count/7

agg_act7 = agg_act7.merge(agg_df_p)
# agg_act7

In [33]:
df_act7_suc = df_act7[df_act7.user_id.isin(suc_id)]
df_act7_sub1 = df_act7[df_act7.user_id.isin(sub1_id)]
df_act7_sub2 = df_act7[df_act7.user_id.isin(sub2_id)]
agg_act7_suc = agg_act7[agg_act7.user_id.isin(suc_id)]
agg_act7_sub1 = agg_act7[agg_act7.user_id.isin(sub1_id)]
agg_act7_sub2 = agg_act7[agg_act7.user_id.isin(sub2_id)]

In [34]:
sum7_suc = agg_act7_suc.mean()
sum7_sub1 = agg_act7_sub1.mean()
sum7_sub2 = agg_act7_sub2.mean()
sum7_df = pd.concat([sum7_suc,sum7_sub1,sum7_sub2],axis=1)
sum7_df.columns = ['suc_w','sub1_w','sub2_w']
# sum7_df

In [35]:
# 定义初次登录当天的聚合统计表
agg_act1 = df_act1.pivot_table(index='user_id',columns='act_target',values='target_id',aggfunc=['count']).fillna(0)
agg_act1.columns = pd.Series(df_act1.act_target.unique()).sort_values()
agg_act1 = agg_act1.reset_index()
agg_act1 = agg_act1.merge(agg_df[['user_id','df_dayofperiod_ptp','act_perc']])

aggs = {
    'dayofperiod':['count','nunique'],
    'action_name':['nunique'],
    'target_cat':['nunique'],
    'act_target':['nunique']
}
agg_df_p = df_act1.groupby('user_id').agg(aggs)
agg_df_p.columns = ['_'.join(col).strip() for col in agg_df_p.columns.values]
agg_df_p = agg_df_p.reset_index()

agg_act1 = agg_act1.merge(agg_df_p)
# agg_act1

In [36]:
df_act1_suc = df_act1[df_act1.user_id.isin(suc_id)]
df_act1_nsuc = df_act1[~df_act1.user_id.isin(suc_id)]
df_act1_sub1 = df_act1[df_act1.user_id.isin(sub1_id)]
df_act1_sub2 = df_act1[df_act1.user_id.isin(sub2_id)]
agg_act1_suc = agg_act1[agg_act1.user_id.isin(suc_id)]
agg_act1_nsuc = agg_act1[~agg_act1.user_id.isin(suc_id)]
agg_act1_sub1 = agg_act1[agg_act1.user_id.isin(sub1_id)]
agg_act1_sub2 = agg_act1[agg_act1.user_id.isin(sub2_id)]

In [37]:
sum1_nsuc = agg_act1_nsuc.mean()
sum1_suc = agg_act1_suc.mean()
sum1_sub1 = agg_act1_sub1.mean()
sum1_sub2 = agg_act1_sub2.mean()
sum1_df = pd.concat([sum1_nsuc,sum1_suc,sum1_sub1,sum1_sub2],axis=1)
sum1_df.columns = ['nsuc_d','suc_d','sub1_d','sub2_d']
# sum1_df

### 活跃情况

从图中可以看到，两类核心人群在新增后的首周内每天的活跃度均比留存用户总体要高。
留存用户的首周的活跃趋势呈现明显减弱的趋势，但是两类核心群体在当周的后半段稳定在四成的活跃人数水平，第一类用户的活跃水平更高。

In [146]:
# 看首次登录后一周内不同日子的情况
# print('留存用户首次登录后一周内的活跃情况')

plt.figure(figsize=(16,10))
plt.subplot(2, 3, 1)
(df_act7_suc.groupby('active_day')['user_id'].count()/len(suc_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 留存用户')

plt.subplot(2, 3, 4)
(df_act7_suc.groupby('active_day')['user_id'].nunique() / len(suc_id)).plot()
plt.ylim(0,1)
plt.title('UV of 留存用户')

# sub1
plt.subplot(2, 3, 2)
(df_act7_sub1.groupby('active_day')['user_id'].count()/len(sub1_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 第一类用户')

plt.subplot(2, 3, 5)
(df_act7_sub1.groupby('active_day')['user_id'].nunique() / len(sub1_id)).plot()
plt.ylim(0,1)
plt.title('UV of 第一类用户')

# sub2
plt.subplot(2, 3, 3)
(df_act7_sub2.groupby('active_day')['user_id'].count()/len(sub2_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 第二类用户')

plt.subplot(2, 3, 6)
(df_act7_sub2.groupby('active_day')['user_id'].nunique() / len(sub2_id)).plot()
plt.ylim(0,1)
plt.title('UV of 第二类用户')

Text(0.5, 1.0, 'UV of 第二类用户')

<Figure size 1152x720 with 6 Axes>

看自然星期日的用户活跃度，第一类用户的低活跃日为周二和周六，而留存总体和第二类用户的低活跃日为周五和周六。

In [139]:
# 看首次登录后一周内不同日子的情况
# print('留存用户首次登录后一周内的活跃情况')

plt.figure(figsize=(16,10))
plt.subplot(2, 3, 1)
(df_act7_suc.groupby('weekday')['user_id'].count()/len(suc_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 留存用户')

plt.subplot(2, 3, 4)
(df_act7_suc.groupby('weekday')['user_id'].nunique() / len(suc_id)).plot()
plt.ylim(0,1)
plt.title('UV of 留存用户')

# sub1
plt.subplot(2, 3, 2)
(df_act7_sub1.groupby('weekday')['user_id'].count()/len(sub1_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 第一类用户')

plt.subplot(2, 3, 5)
(df_act7_sub1.groupby('weekday')['user_id'].nunique() / len(sub1_id)).plot()
plt.ylim(0,1)
plt.title('UV of 第一类用户')

# sub2
plt.subplot(2, 3, 3)
(df_act7_sub2.groupby('weekday')['user_id'].count()/len(sub2_id)).plot(color='C3')
plt.ylim(0,10)
plt.title('PV of 第二类用户')

plt.subplot(2, 3, 6)
(df_act7_sub2.groupby('weekday')['user_id'].nunique() / len(sub2_id)).plot()
plt.ylim(0,1)
plt.title('UV of 第二类用户')

Text(0.5, 1.0, 'UV of 第二类用户')

<Figure size 1152x720 with 6 Axes>

从一天中活跃时间段维度来对比，社区用户基本都是在一天中的10点、15点和20点三个时间点附近最为活跃，活跃时间峰值都是15点。
第二类用户的整体模式与留存用户整体更接近。
比较来看，第一类用户在更晚时间段(22点后)的活跃度下降明显，猜测该类用户中是否由更多的学生组成？

In [140]:
# 一天中不同时间的情况
# print('一天中不同时间的日均活跃情况')

plt.figure(figsize=(16,10))
plt.subplot(2, 3, 1)
(df_act7_suc.groupby('hour')['user_id'].count()/len(suc_id)).plot(color='C3')
plt.ylim(0,4)
plt.title('留存用户 一周人均PV')

plt.subplot(2, 3, 4)
(df_act7_suc.groupby('hour')['user_id'].nunique() / len(suc_id)).plot()
plt.ylim(0,0.7)
plt.title('UV of 留存用户')

# sub1
plt.subplot(2, 3, 2)
(df_act7_sub1.groupby('hour')['user_id'].count()/len(sub1_id)).plot(color='C3')
plt.ylim(0,4)
plt.title('第一类用户 一周人均PV')

plt.subplot(2, 3, 5)
(df_act7_sub1.groupby('hour')['user_id'].nunique() / len(sub1_id)).plot()
plt.ylim(0,0.7)
plt.title('UV of 第一类用户')

# sub2
plt.subplot(2, 3, 3)
(df_act7_sub2.groupby('hour')['user_id'].count()/len(sub2_id)).plot(color='C3')
plt.ylim(0,4)
plt.title('第二类用户 一周人均PV')

plt.subplot(2, 3, 6)
(df_act7_sub2.groupby('hour')['user_id'].nunique() / len(sub2_id)).plot()
plt.ylim(0,0.7)
plt.title('UV of 第二类用户')


Text(0.5, 1.0, 'UV of 第二类用户')

<Figure size 1152x720 with 6 Axes>

### 行为模式

那么不同类型的用户在初次登录社区后的第一周都做了哪些行为呢？
尤其是两类核心的焦点用户群，是否有某项行为导致了他们相比普通用户来说更为活跃呢？

In [22]:
print('留存用户在初次使用第一周的行为比例：')
print('--------------------------------')
(df_act7_suc.pivot_table(index='target_cat',columns='action_name',values='user_id'\
,aggfunc=['nunique'])/len(suc_id)).round(3).fillna('-')

留存用户在初次使用第一周的行为比例：
--------------------------------


nunique                                         
action_name     buy   comp   mark reward share   view   vote
target_cat                                                  
0                 -      -      -  0.226     -      -      -
1                 -      -      -  0.011     -      -      -
5                 -      -  0.095      -  0.28  0.748  0.055
6             0.499      -  0.046      -     -  0.741  0.021
A                 -  0.015      -      -     -      -      -
B                 -  0.001      -      -     -      -      -
C                 -  0.013      -      -     -      -      -
D                 -  0.043      -      -     -      -      -
E                 -  0.058      -      -     -      -      -

In [23]:
print('第一类用户在初次使用第一周的行为比例：')
print('--------------------------------')
(df_act7_sub1.pivot_table(index='target_cat',columns='action_name',values='user_id'\
,aggfunc=['nunique'])/len(sub1_id)).round(3).fillna('-')

第一类用户在初次使用第一周的行为比例：
--------------------------------


nunique                                          
action_name     buy   comp   mark reward  share   view   vote
target_cat                                                   
0                 -      -      -  0.394      -      -      -
1                 -      -      -   0.07      -      -      -
5                 -      -  0.155      -  0.408  0.831  0.085
6             0.479      -  0.042      -      -  0.789      -
A                 -  0.042      -      -      -      -      -
C                 -  0.014      -      -      -      -      -
D                 -  0.085      -      -      -      -      -
E                 -  0.085      -      -      -      -      -

In [24]:
print('第二类用户在初次使用第一周的行为比例：')
print('--------------------------------')
(df_act7_sub2.pivot_table(index='target_cat',columns='action_name',values='user_id'\
,aggfunc=['nunique'])/len(sub2_id)).round(3).fillna('-')

第二类用户在初次使用第一周的行为比例：
--------------------------------


nunique                                          
action_name     buy   comp   mark reward  share   view   vote
target_cat                                                   
0                 -      -      -  0.398      -      -      -
1                 -      -      -  0.016      -      -      -
5                 -      -  0.164      -  0.428  0.895  0.119
6              0.44      -  0.047      -      -  0.755  0.026
A                 -  0.043      -      -      -      -      -
C                 -  0.021      -      -      -      -      -
D                 -  0.032      -      -      -      -      -
E                 -  0.062      -      -      -      -      -

In [108]:
# 不同行为-对象组合的情况
print('行为流量总量分布')
print('----------------------------------------------')

plt.figure(figsize=(25,6))
plt.subplot(1, 3, 1)
# (df.groupby('act_target')['user_id'].count()/df.groupby('act_target').date.nunique()).plot.bar(color='C3')
df.groupby('act_target')['user_id'].count().plot.bar(color='C3')
plt.title('总体PV')

plt.subplot(1, 3, 2)
# (df.groupby('act_target')['user_id'].nunique() / df.groupby('act_target').date.nunique()).plot.bar()
df.groupby('act_target')['user_id'].nunique().plot.bar()
plt.title('总体UV')

plt.subplot(1, 3, 3)
# (df.groupby('act_target')['target_id'].nunique()/df.groupby('act_target').date.nunique()).plot.bar(color='orange')
df.groupby('act_target')['target_id'].nunique().plot.bar(color='orange')
plt.title('总体内容对象数量')
plt.show()

行为流量总量分布
----------------------------------------------


<Figure size 1800x432 with 3 Axes>

结合上图中各类操作的总量与UV、对象数量水平来分析

第一类用户使用首周的行为特点：
* **reward过0类**内容对象的比例更高：0.394 vs 0.226
* 针对大类5的内容对象，**share**和view的比例更高：share 0.408 vs 0.28, view 0.831 vs 0.748
* **comp**行为对象为**A、D和E类**的比例轻微提高

第二类用户使用首周的行为特点：
* **reward过0类**内容对象的比例更高：0.398 vs 0.226
* 针对大类5的内容对象，**share**和view的比例更高：share 0.428 vs 0.28, view 0.895 vs 0.748；另外**vote的比例提升更明显**，0.119 vs 0.055
*** comp**行为对象为**A类**的比例轻微提高

进一步尝试加入更多的行为统计指标：
* nsuc_d 代表一次性用户的首日行为
* suc为留存用户总体；sub1和sub2分别代表第一类和第二类用户
* d为新增首日，w代表新增后首周内的行为
* df_dayofperiod_ptp：用户的生命周期长度
* dayofperiod_count：对应时间区间内的日均行为个数
* act_target_nunique：行为与对象大类交叉类型的涉及数
* action_name_nunique：行为类型的涉及数
* target_cat_nunique：内容对象大类的涉及数
* w_act_perc：首周内活跃天数的比例
* act_perc：生命周期对应内总体的活跃天数比例
* 行为类型 X 对象大类：每个行为类型在对应时间区间内的日均操作次数

In [38]:
ana_df = pd.concat([sum1_df,sum7_df],axis=1).sort_values(by='sub1_w',ascending=False).round(3)
ana_df['max'] = ana_df.max(axis=1)
ana_df

,nsuc_d,suc_d,sub1_d,sub2_d,suc_w,sub1_w,sub2_w,max
df_dayofperiod_ptp,0.000,39.829,15.944,117.128,39.829,15.944,117.128,117.128
dayofperiod_count,4.236,7.811,10.310,8.059,2.428,5.076,4.104,10.310
dayofperiod_nunique,1.000,1.000,1.000,1.000,2.388,4.521,3.527,4.521
act_target_nunique,1.974,2.421,2.676,2.568,2.853,3.479,3.446,3.479
view_5,1.263,3.163,4.592,4.138,1.087,2.706,2.333,4.592
action_name_nunique,1.712,1.958,2.141,2.023,2.253,2.690,2.667,2.690
target_cat_nunique,1.355,1.622,1.803,1.813,1.857,2.310,2.222,2.310
view_6,1.951,3.241,3.577,2.317,0.953,1.483,1.043,3.577
w_act_perc,NaN,NaN,NaN,NaN,0.341,0.646,0.504,0.646
act_perc,1.000,0.398,0.642,0.331,0.398,0.642,0.331,1.000


通过对比各行的数据，可以总结发现：

* 从view第5和6类内容的比例来看，第一类用户在此行为上的活跃度非常高，第二类用户略低于第一类的水平，但都比留存用户总体明显更高。
* 第一类用户首日有56.3%的人做了操作 mark第5类内容。
* 第二类用户buy第6类内容的比例反而不高。
* 两类用户对第5类用户的share比例差不多，均高于用户总体。
* 第二类用户首日就有41%的人做了reward第0类内容的行为，首周内的水平与第一类用户接近。


总体来看，两类核心用户的更多操作对象还是集中在第5类内容上，其次是第6类。

### 相关性分析
进一步，计算相关系数，看看哪个行为指标与总体生命周期的关系更大

In [106]:
cor7_suc = agg_act7_suc.corr().df_dayofperiod_ptp
cor7_sub1 = agg_act7_sub1.corr().df_dayofperiod_ptp
cor7_sub2 = agg_act7_sub2.corr().df_dayofperiod_ptp
cor7_df = pd.concat([cor7_suc,cor7_sub1,cor7_sub2],axis=1)
cor7_df.columns = ['suc_w','sub1_w','sub2_w']
plt_df = cor7_df.sort_values(by='sub2_w',ascending=False).round(3)[2:]

print('首周活跃指标：与总体生命周期(df_dayofperiod_ptp)的各指标相关系数：按第二类用户列排序')
print('------------------------------------------------------------------')
fig,ax = plt.subplots()
plt.rcParams["figure.figsize"] = (10,6)
ax.plot(plt_df)
ax.set_xticklabels(plt_df.index.to_list(),rotation=90)
plt.axhline(y=0,ls='--',color='C3')
plt.legend(['留存用户','第一类','第二类'])
plt.show()

首周活跃指标：与总体生命周期(df_dayofperiod_ptp)的各指标相关系数：按第二类用户列排序
------------------------------------------------------------------


<Figure size 720x432 with 1 Axes>

In [107]:
# cor1_nsuc = agg_act1_nsuc.corr().df_dayofperiod_ptp
cor1_suc = agg_act1_suc.corr().df_dayofperiod_ptp
cor1_sub1 = agg_act1_sub1.corr().df_dayofperiod_ptp
cor1_sub2 = agg_act1_sub2.corr().df_dayofperiod_ptp
cor1_df = pd.concat([cor1_suc,cor1_sub1,cor1_sub2],axis=1)
cor1_df.columns = ['suc_d','sub1_d','sub2_d']
plt_df = cor1_df.sort_values(by='sub2_d',ascending=False).round(3)[2:]

print('首日活跃指标：与总体生命周期(df_dayofperiod_ptp)的各指标相关系数：按第二类用户列排序')
print('------------------------------------------------------------------')
fig,ax = plt.subplots()
plt.rcParams["figure.figsize"] = (10,6)
ax.plot(plt_df)
ax.set_xticklabels(plt_df.index.to_list(),rotation=90)
plt.axhline(y=0,ls='--',color='C3')
plt.legend(['留存用户','第一类','第二类'])
plt.show()

首日活跃指标：与总体生命周期(df_dayofperiod_ptp)的各指标相关系数：按第二类用户列排序
------------------------------------------------------------------


<Figure size 720x432 with 1 Axes>

可以猜测：
* **mark在6类 和 vote在5类** 上的动作可能意味着用户短期内会频繁的登录社区，与第一类用户的生命周期高度相关
* **reward在0类 和 comp在C类**上的行为会有助于用户的更长期存在，是与第二类用户生命周期长度最为正相关的行为指标


## 总结与建议

- 体验的模块越多，用户越会短期活跃甚至是长期存留：可以建立初期登录时有趣的新手任务或积分体系，促使用户在接触社区的一开始可以更多地了解社区

- **第5类内容对象**的角色更像是社区的“流量担当”，数量众多，且对其的vote行为与活跃和留存正相关：同样可以考虑与积分体系挂钩，同时尝试推出激励用户点赞的活动，让用户也可以享受到点赞给自己带来的好处

- **第6类内容对象**更有些“骨干力量”的架势，数量不多但是由它带来的UV却是最多的：可见保证第6类内容的质量是重中之重

- 基于对第二类用户的分析结果，对于初期有**对0类内容reward**或者**对C类对象comp**行为的用户，可以重点关注：可以尝试对他们做更多地用户研究，多倾听他们的需求和反馈，对于改进社区体验大有帮助

- 数据收集时，尝试加入能够判断用户动机的信息：对于更有效的用户分层会非常有用

## 更多思考

- 知识内容类的社区用户，易存类“排他性”的心理，即出于“自己想会但是怕别人也会”的心理，可能不会产生很多的分享动作。
   但是会有一类用户，出于塑造“个人品牌”的动机，会乐于创作，也会乐于将自己的成果对外分享。
   类似于知乎的大V创作者，可以为社区形成优秀的内容，同时也能够高效地吸引更多用户加入。

- 学习型的知识社区可能还存在的一个先天的劣势，就是主营服务内容是“反人性”的。
   学习-成长-进步，这类的活动本身就需要个人坚强的品质来支撑和维持。
	 因此，社区的使用体验上，如果可以加入更多的趣味性，可能会对用户长期留存有益。

- 有一个具体的指标也很值得产品或UX部门进行分析，就是一天或一次会话内登录同一页面的次数。
   这一点思考缘于个人体验：使用某一个网站的时候，因为很难找到想找的内容，只能从我收藏的入口不停地进去寻找。
	 这一指标过高可能揭示了网站设计上层级或逻辑的问题。

---